# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

In [2]:
# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

In [3]:
# Read School Data File and store into Pandas Data Frames
# Removing ID columns, seems unnecessary for now
school_data = pd.read_csv(school_data_to_load).drop(columns=['School ID'])
school_data.head()

,school_name,type,size,budget
0,Huang High School,District,2917,1910635
1,Figueroa High School,District,2949,1884411
2,Shelton High School,Charter,1761,1056600
3,Hernandez High School,District,4635,3022020
4,Griffin High School,Charter,1468,917500


In [4]:
# Read Student Data File and store into Pandas Data Frames
# Removing ID columns, seems unnecessary for now
student_data = pd.read_csv(student_data_to_load).drop(columns=['Student ID'])
student_data.head()

,student_name,gender,grade,school_name,reading_score,math_score
0,Paul Bradley,M,9th,Huang High School,66,79
1,Victor Smith,M,12th,Huang High School,94,61
2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,Bonnie Ray,F,9th,Huang High School,97,84


In [5]:
# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,student_name,gender,grade,school_name,reading_score,math_score,type,size,budget
0,Paul Bradley,M,9th,Huang High School,66,79,District,2917,1910635
1,Victor Smith,M,12th,Huang High School,94,61,District,2917,1910635
2,Kevin Rodriguez,M,12th,Huang High School,90,60,District,2917,1910635
3,Dr. Richard Scott,M,12th,Huang High School,67,58,District,2917,1910635
4,Bonnie Ray,F,9th,Huang High School,97,84,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [8]:
# calculate values for district summary
tot_sch = len(school_data.index)
tot_stu = len(student_data.index)

tot_bud = school_data['budget'].sum()

avg_math = student_data['math_score'].mean()
avg_read = student_data['reading_score'].mean()

# using mean of count, since all numbers collected are the same anyway
pass_math = student_data[student_data['math_score'] >= 70].count().mean()/tot_stu*100
pass_read = student_data[student_data['reading_score'] >= 70].count().mean()/tot_stu*100
avg_score = (pass_math + pass_read)/2

In [11]:
# store in dataframe and format results
district_summary_df = pd.DataFrame({'total schools':[tot_sch],
                                    'total students':[tot_stu],
                                    'total budget':[tot_bud],
                                    'average math score':[avg_math],
                                    'average reading score':[avg_read],
                                    '% passing math':[pass_math],
                                    '% passing reading':[pass_read],
                                    '% overall passing':[avg_score]})

# format results using map command
district_summary_df['total schools'] = district_summary_df['total schools'].map("{:,}".format)
district_summary_df['total students'] = district_summary_df['total students'].map("{:,}".format)
district_summary_df['total budget'] = district_summary_df['total budget'].map("${:,.2f}".format)

district_summary_df.head()

,total schools,total students,total budget,average math score,average reading score,% passing math,% passing reading,% overall passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [14]:
# create sorted dataframe for understanding
schools = school_data.set_index('school_name').sort_index()

# calculate values for school summary
sch_type = schools['type']
sch_size = schools['size']

sch_bud = schools['budget']
stu_bud = sch_bud/sch_size

avg_math = student_data.groupby('school_name')['math_score'].mean()
avg_read = student_data.groupby('school_name')['reading_score'].mean()

# count passing students after groupby, for count per school
pass_math = student_data[student_data['math_score'] >= 70].groupby('school_name').count()['math_score']/sch_size*100
pass_read = student_data[student_data['reading_score'] >= 70].groupby('school_name').count()['math_score']/sch_size*100
avg_score = (pass_math + pass_read)/2

In [15]:
# store in dataframe and format results
school_summary_df = pd.DataFrame({'school type':sch_type,
                                  'total students':sch_size,
                                  'school budget':sch_bud,
                                  'student budget':stu_bud,
                                  'average math score':avg_math,
                                  'average reading score':avg_read,
                                  '% passing math':pass_math,
                                  '% passing reading':pass_read,
                                  '% overall passing':avg_score})

# format results using map command
school_summary_df['total students'] = school_summary_df['total students'].map("{:,}".format)
school_summary_df['student budget'] = school_summary_df['student budget'].map("${:,.2f}".format)
school_summary_df['school budget'] = school_summary_df['school budget'].map("${:,.2f}".format)

school_summary_df.head()

,school type,total students,school budget,student budget,average math score,average reading score,% passing math,% passing reading,% overall passing
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [16]:
top_performing = school_summary_df.sort_values(by=['% overall passing'],ascending=False)
top_performing.head()

,school type,total students,school budget,student budget,average math score,average reading score,% passing math,% passing reading,% overall passing
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [17]:
bottom_performing = school_summary_df.sort_values(by=['% overall passing'],ascending=True)
bottom_performing.head()

,school type,total students,school budget,student budget,average math score,average reading score,% passing math,% passing reading,% overall passing
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [18]:
grade_9 = student_data[student_data['grade'] == '9th'].groupby('school_name')['math_score'].mean()
grade_10 = student_data[student_data['grade'] == '10th'].groupby('school_name')['math_score'].mean()
grade_11 = student_data[student_data['grade'] == '11th'].groupby('school_name')['math_score'].mean()
grade_12 = student_data[student_data['grade'] == '12th'].groupby('school_name')['math_score'].mean()

math_by_grade = pd.DataFrame({'9th':grade_9,
                              '10th':grade_10,
                              '11th':grade_11,
                              '12th':grade_12})
math_by_grade.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [19]:
grade_9 = student_data[student_data['grade'] == '9th'].groupby('school_name')['reading_score'].mean()
grade_10 = student_data[student_data['grade'] == '10th'].groupby('school_name')['reading_score'].mean()
grade_11 = student_data[student_data['grade'] == '11th'].groupby('school_name')['reading_score'].mean()
grade_12 = student_data[student_data['grade'] == '12th'].groupby('school_name')['reading_score'].mean()

reading_by_grade = pd.DataFrame({'9th':grade_9,
                                 '10th':grade_10,
                                 '11th':grade_11,
                                 '12th':grade_12})
reading_by_grade.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [24]:
# Sample bins. Feel free to create your own bins.
bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
print('min: ' + str(school_summary_df['student budget'].min()))
print('max: ' + str(school_summary_df['student budget'].max()))

min: 578.0
max: 655.0


In [25]:
# need the unformatted budget for bins
# .map("${:,.2f}".format) converts to str
school_summary_df['student budget'] = stu_bud
school_summary_df["spending ranges"] = pd.cut(school_summary_df["student budget"], bins, labels=group_names)

# RE-format results using map command
school_summary_df['student budget'] = school_summary_df['student budget'].map("${:,.2f}".format)

# find means of numerical columns
scores_by_spending = school_summary_df.groupby('spending ranges').mean()
scores_by_spending.head()

,average math score,average reading score,% passing math,% passing reading,% overall passing
spending ranges,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Scores by School Type

* Perform the same operations as above, based on school type.